# 2

In [14]:
import torch
import pandas as pd
#


from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from google.colab import drive
drive.mount('/content/drive')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import mean_absolute_error, r2_score
import numpy as np

from sklearn.model_selection import KFold




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
data = pd.read_csv('drive/MyDrive/advData/cars.csv')

In [16]:
class RidgeRegression:
    def __init__(self, n_features, learning_rate=0.01, regularization_param=0.1, epochs=1000):
        self.model = torch.nn.Linear(n_features, 1, bias=True)
        self.learning_rate = learning_rate
        self.regularization_param = regularization_param
        self.epochs = epochs
        self.criterion = torch.nn.MSELoss()
        self.optimizer = torch.optim.SGD(self.model.parameters(), lr=self.learning_rate)

    def fit(self, X, y):
        X = torch.tensor(X, dtype=torch.float32)
        # Convert y to a numpy array if it's a pandas Series, then to a PyTorch tensor
        y = torch.tensor(y.values if isinstance(y, pd.Series) else y, dtype=torch.float32)
        y = y.view(-1, 1)  # Ensure y is a column vector
        for _ in range(self.epochs):
            self.optimizer.zero_grad()
            outputs = self.model(X)
            loss = self.criterion(outputs, y) + self.regularization_param * torch.sum(self.model.weight ** 2)
            loss.backward()
            self.optimizer.step()


    def predict(self, X):
        with torch.no_grad():
            X = torch.tensor(X, dtype=torch.float32)
            predictions = self.model(X)
            return predictions.numpy()


In [17]:
# Select features and target variable
X = data[['CYL', 'ENG', 'WGT']]
y = data['MPG']

# Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

from sklearn.preprocessing import PolynomialFeatures

# Create polynomial features
poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly.fit_transform(X_scaled)

# Split the data into training and testing sets with polynomial features
X_train_poly, X_test_poly, y_train, y_test = train_test_split(X_poly, y, test_size=0.2, random_state=42)

# Initialize and train the Ridge Regression model with polynomial features
ridge_model_poly = RidgeRegression(n_features=X_train_poly.shape[1], learning_rate=0.01, regularization_param=0.1, epochs=1000)
ridge_model_poly.fit(X_train_poly, y_train)

# Make predictions on the test set with polynomial features
predictions_poly = ridge_model_poly.predict(X_test_poly)

# Calculate new metrics
mse_poly = mean_squared_error(y_test, predictions_poly)
rmse_poly = np.sqrt(mse_poly)
mae_poly = mean_absolute_error(y_test, predictions_poly)
r_squared_poly = r2_score(y_test, predictions_poly)

print(f"Polynomial Features - MSE: {mse_poly}, RMSE: {rmse_poly}, MAE: {mae_poly}, R-squared: {r_squared_poly}")


Polynomial Features - MSE: 17.276900393041984, RMSE: 4.156549096671659, MAE: 3.2203683612400487, R-squared: 0.6615065934057782


In [18]:

# Gaussian Kernel
def Gaussian(x):
  return np.where(np.abs(x)>4,0,1/(np.sqrt(2*np.pi))*np.exp(-1/2*x**2))

# Quartic Kernel
def Quartic(x):
  return np.where(np.abs(x)>1,0,15/16*(1-np.abs(x)**2)**2)

# Epanechnikov Kernel
def Epanechnikov(x):
  return np.where(np.abs(x)>1,0,3/4*(1-np.abs(x)**2))
  # this is the correct vectorized version
def Tricubic(x):
  return np.where(np.abs(x)>1,0,(1-np.abs(x)**3)**3)

# 3

In [29]:




def dist1(u,v):
  if len(v.shape)==1:
    v = v.reshape(1,-1)
  d = np.array([np.sqrt(np.sum((u-v[i])**2,axis=1)) for i in range(len(v))])
  return d

# how about the case when v is a matrix with 5 columns?
def dist2(u,v):
  D = []
  if len(v.shape)==1:
    v = v.reshape(1,-1)
  # we would like all the pairwise combinations if u and v are matrices
  # we could avoid two for loops if we consider broadcasting
  for rowj in v:
    D.append(np.sqrt(np.sum((u-rowj)**2,axis=1)))
  return np.array(D).T


def kernel_function(xi,x0,kern, tau):
    return kern(dist1(xi,x0)/(2*tau))

def weights_matrix(x,x_new,kern,tau):
  if np.isscalar(x_new):
    return kernel_function(x,x_new,kern,tau)
  else:
    n = len(x_new)
    return np.array([kernel_function(x,x_new[i],kern,tau) for i in range(n)])

def weight_function(u,v,kern=Gaussian,tau=0.5):
    return kern(dist2(u,v)/(2*tau))

In [37]:
class Lowess:
    def __init__(self, kernel = Gaussian, tau=0.05):
        self.kernel = kernel
        self.tau = tau

    def fit(self, x, y):
        kernel = self.kernel
        tau = self.tau
        self.xtrain_ = x
        self.yhat_ = y

    def predict(self, x_new):
        check_is_fitted(self)
        x = self.xtrain_
        y = self.yhat_
        lm = linear_model.Ridge(alpha=0.001)
        w = weight_function(x,x_new,self.kernel,self.tau)

        if np.isscalar(x_new):
          #print(x)
          lm.fit(np.diag(w)@(x.reshape(-1,1)),np.diag(w)@(y.reshape(-1,1)))
          yest = lm.predict([[x_new]])[0][0]
        else:
          n = len(x_new)
          yest_test = np.zeros(n)
          #Looping through all x-points
          for i in range(n):
            #print(x)
            #print(x.shape)
            lm.fit(np.diag(w[:,i])@x,np.diag(w[:,i])@y)
            yest_test[i] = lm.predict(x_new[i].reshape(1,-1))
        return yest_test

In [39]:




import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.utils.validation import check_is_fitted
from sklearn import linear_model



# Linear regression model for Lowess

#Scalar
scale = MinMaxScaler()
#loading data
X = data.loc[:, 'CYL':'WGT'].values  # Multiple features
y = data['MPG'].values

mse_lwr = []
r_squared = []

ep_mse=[]
r_squaredE=[]

#split train test
kf = KFold(n_splits=10,shuffle=True,random_state=1234)
lowess_model = Lowess(kernel=Gaussian, tau=0.05)
model_lw = Lowess(kernel= Epanechnikov,tau=0.14)

for idxtrain, idxtest in kf.split(X):
    xtrain, ytrain = X[idxtrain], y[idxtrain]
    xtest, ytest = X[idxtest], y[idxtest]

    # Scale the features
    xtrain_scaled = scale.fit_transform(xtrain)
    xtest_scaled = scale.transform(xtest)

    # Fit the Lowess model and make predictions
    lowess_model.fit(xtrain_scaled, ytrain)
    predictions = lowess_model.predict(xtest_scaled)

    # Calculate and store MSE and R-squared for this fold
    mse_lwr.append(mean_squared_error(ytest, predictions))
    r_squared.append(r2_score(ytest, predictions))

    # Fit the Lowess model and make predictions
    model_lw.fit(xtrain_scaled, ytrain)
    predictions = model_lw.predict(xtest_scaled)

    # Calculate and store MSE and R-squared for this fold
    ep_mse.append(mean_squared_error(ytest, predictions))
    r_squaredE.append(r2_score(ytest, predictions))

print('The Cross-validated Mean Squared Error for Locally Weighted Regression is:', np.mean(mse_lwr))
print('The mean R-squared:', np.mean(r_squared))


print('The Cross-validated Mean Squared Error for Locally Weighted Regression is:', np.mean(ep_mse))
print('The mean R-squared:', np.mean(r_squaredE))


The Cross-validated Mean Squared Error for Locally Weighted Regression is: 16.781484620398693
The mean R-squared: 0.7158283305017227
The Cross-validated Mean Squared Error for Locally Weighted Regression is: 16.455751824170868
The mean R-squared: 0.7204516544343604
